In [ ]:
import start
sentence = start.sentence_clean("❤️投資情報収集 ❤️動画編集 ❤️瞑想とヨガ 今日もゆるゆる😌☀️そろそろBTC発射かな…⤴️か⤵️か一気に行きそう😳今日日曜日だし。 今日はお天気もいいし❤️秋晴れ～気持ちいい～😊 しっかり季節の気を取り入れます😄🌈  #今日の積み上げ #ビットコイン #動画編集 #投資  #瞑想  https://t.co/lCbClcjGsm")
print(sentence)
sentence = start.sentence_tokenizer(sentence)
print()
print(sentence)


In [ ]:
import nltk
from nltk.corpus import stopwords
import unicodedata

#nltk.download('all')
text = "I guess Solana maxis were right in comparing Solana to Facebook"
text = unicodedata.normalize("NFKC", text)
# 分かち書き
morph = nltk.word_tokenize(text)
pos = nltk.pos_tag(morph)
# 品詞を取得
for i in pos:
    print(i[0],i[1])

target_pos = ["NN","NNP"]
tag = [f for f in pos if (pos in target_pos)]
#token_list = [t.lower() for t in tag]
#result = " ".join(token_list)
print(tag)

#print(tag)
# 名詞,動詞,形容詞のみに絞り込み
#target_pos = ["名詞","形容詞"]
#target_pos = ["名詞"]
#token_list = [t for t, p in zip(token_list, pos) if (p in target_pos)]

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = "91d0454ee5244ea88056d7e79b73b9d2"
endpoint = "https://computerkun.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

'''
OCR: Read File using the Read API, extract text - remote
This example will extract text in an image, then print results, line by line.
This API call can also extract handwriting style text (not shown).
'''
print("===== Read File - remote =====")
# Get an image with text
read_image_url = "https://stat.ameba.jp/user_images/20140221/16/koumuinsiken-yobikou/86/3b/p/o0471068512853433393.png"

# Call API with URL and raw response (allows you to get the operation location)
read_response = computervision_client.read(read_image_url,  raw=True)

# Get the operation location (URL with an ID at the end) from the response
read_operation_location = read_response.headers["Operation-Location"]
# Grab the ID from the URL
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Print the detected text, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()

In [ ]:
'''
OCR: Read File using the Read API, extract text - local
This example extracts text from a local image, then prints results.
This API call can also recognize remote image text (shown in next example, Read File - remote).
'''
print("===== Read File - local =====")

images_folder = "/workspace/png"
# Get image path
read_image_path = os.path.join (images_folder, "bunsyo.png")
# Open the image
read_image = open(read_image_path, "rb")

# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(10)

# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            print(line.text)
            print(line.bounding_box)
print()
'''
END - Read File - local
'''

print("End of Computer Vision quickstart.")